# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lautoka,-17.6167,177.4667,27.92,57,40,6.69,FJ,1691038376
1,1,qaqortoq,60.7167,-46.0333,5.49,89,93,2.35,GL,1691038376
2,2,rice lake,45.5061,-91.7382,22.99,100,0,0.00,US,1691038224
3,3,hermanus,-34.4187,19.2345,13.78,46,96,4.64,ZA,1691038377
4,4,ishigaki,24.3448,124.1572,26.07,94,75,12.35,JP,1691038378


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Create the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=1200,
    frame_height=600,
    size="Humidity",
    scale=0.9,
    color="City",
    cmap="coolwarm",  
    line_color="black", 
    hover_cols=["City", "Humidity"], 
    title="City Humidity on World Map"
)

# Display the map plot
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
# Temperature between 20 and 25 degrees Celsius and humidity less than 70
filtered_city_data_df = city_data_df[
    (city_data_df['Max Temp'] >= 20) &
    (city_data_df['Max Temp'] <= 25) &
    (city_data_df['Humidity'] < 70)
]

# Drop any rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()

# Display sample data
filtered_city_data_df.head(15)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
23,23,rikitea,-23.1203,-134.9692,20.41,65,77,9.23,PF,1666108244
27,27,bambous virieux,-20.3428,57.7575,22.14,68,75,3.60,MU,1666108246
34,34,naberera,-4.2000,36.9333,21.12,54,94,6.38,TZ,1666108252
53,53,lasa,46.6166,10.7002,23.49,68,11,2.01,IT,1666108019
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
64,64,atuona,-9.8000,-139.0333,24.69,69,1,7.05,PF,1666108268
79,79,mahebourg,-20.4081,57.7000,22.16,68,75,3.60,MU,1666108278
86,86,rafaela,-31.2503,-61.4867,21.10,42,100,1.79,AR,1666108109


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Check if the DataFrame city_data_df is already loaded, if not, load it from the CSV file
if 'city_data_df' not in locals():
    # Import API key
    from api_keys import geoapify_key
    
    # Load the CSV file created in Part 1 into a Pandas DataFrame
    csv_file = Path('output_data', 'cities.csv')
    city_data_df = pd.read_csv(csv_file)

# Use the Pandas copy function to create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(17)



,City,Country,Lat,Lng,Humidity,Hotel Name
0,lautoka,FJ,-17.6167,177.4667,57,
1,qaqortoq,GL,60.7167,-46.0333,89,
2,rice lake,US,45.5061,-91.7382,100,
3,hermanus,ZA,-34.4187,19.2345,46,
4,ishigaki,JP,24.3448,124.1572,94,
5,tazacorte,ES,28.6290,-17.9293,73,
6,grytviken,GS,-54.2811,-36.5092,95,
7,bilibino,RU,68.0546,166.4372,94,
8,reggane,DZ,26.7158,0.1714,10,
9,tiksi,RU,71.6872,128.8694,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "apiKey": geoapify_key,
    "type": "hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{longitude},{latitude}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    data = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(17)


Starting hotel search
lautoka - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
rice lake - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
ishigaki - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
reggane - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
coquimbo - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
rio grande - nearest hotel: No hotel found
antofagasta - nearest hotel: No hotel found
weiser - nearest hotel: No hotel found
kone - nearest hotel: No hotel found
gazli - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
yuzhno-kurilsk - nearest hotel: No hotel found
vadso - nearest hotel: No hotel found
adamst

ibiza - nearest hotel: No hotel found
udachny - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
racine - nearest hotel: No hotel found
rajula - nearest hotel: No hotel found
hue - nearest hotel: No hotel found
tadine - nearest hotel: No hotel found
zhangjiakou - nearest hotel: No hotel found
labytnangi - nearest hotel: No hotel found
victorica - nearest hotel: No hotel found
katsuren-haebaru - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
chinde - nearest hotel: No hotel found
machakos - nearest hotel: No hotel found
las terrenas - nearest hotel: No hotel found
ulsteinvik - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
galliano - nearest hotel: No hotel found
mhamid - nearest hotel: No hotel found
bangassou - nearest hotel: No hotel found
betio village - nearest hotel: No hotel found
san rafael - nearest hotel: No hotel found
mayya - nearest hotel: No hotel found
bantay - nearest hotel: No hotel found
puerto armuel

fisterra - nearest hotel: No hotel found
st. paul - nearest hotel: No hotel found
sibut - nearest hotel: No hotel found
vorkuta - nearest hotel: No hotel found
al mijlad - nearest hotel: No hotel found
sinabang - nearest hotel: No hotel found
southeast arcadia - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
necochea - nearest hotel: No hotel found
north kingstown - nearest hotel: No hotel found
hauterive - nearest hotel: No hotel found
trogir - nearest hotel: No hotel found
puerto madryn - nearest hotel: No hotel found
rio preto da eva - nearest hotel: No hotel found
kitimat - nearest hotel: No hotel found
visakhapatnam - nearest hotel: No hotel found
ambon - nearest hotel: No hotel found
neman - nearest hotel: No hotel found
port lincoln - nearest hotel: No hotel found
warman - nearest hotel: No hotel found
sao gabriel da cachoeira - nearest hotel: No hotel found
gaspe - nearest hotel: No hotel found
iwanai - nearest hotel: No hotel found
tralee - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
0,lautoka,FJ,-17.6167,177.4667,57,No hotel found
1,qaqortoq,GL,60.7167,-46.0333,89,No hotel found
2,rice lake,US,45.5061,-91.7382,100,No hotel found
3,hermanus,ZA,-34.4187,19.2345,46,No hotel found
4,ishigaki,JP,24.3448,124.1572,94,No hotel found
5,tazacorte,ES,28.6290,-17.9293,73,No hotel found
6,grytviken,GS,-54.2811,-36.5092,95,No hotel found
7,bilibino,RU,68.0546,166.4372,94,No hotel found
8,reggane,DZ,26.7158,0.1714,10,No hotel found
9,tiksi,RU,71.6872,128.8694,89,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=1200,
    frame_height=600,
    size="Humidity",
    scale=0.95, 
    color="City",
    cmap="coolwarm",  
    line_color="black",  
    hover_cols=["City", "Country", "Humidity", "Hotel Name"], 
    title="City Humidity on World Map"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)